In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [35]:
account = pd.read_csv('anomaly-detection_anomaly-detection/faccount.txt')
account.set_index('ACCTID',inplace=True)
account.head()

,ACCTTYPE,ACCTNAME,FIRSTNAME,SURNAME,DOB,ACCTCREATED,ACCTRISK,ADDRESS,ADDRESS_LAT,ADDRESS_LON,CATEGORY
ACCTID,,,,,,,,,,,
800039,BUSINESS,Young Supermarket,Kevin,Young,1982-02-23T00:00:00,2018-09-20T00:00:00,MEDIUM,BRISBANE,-27.512179,153.073105,Supermarket
800040,BUSINESS,Williams Grocery,Lindsay,Williams,1959-02-20T00:00:00,2018-02-24T00:00:00,MEDIUM,BRISBANE,-27.504303,153.073470,Grocery
800041,CREDIT,Myers Supermarket,Lisa,Myers,1982-09-04T00:00:00,2018-01-11T00:00:00,HIGH,BRISBANE,-27.484970,153.097588,Supermarket
800042,BUSINESS,Martinez Supermarket,Brittany,Martinez,1976-10-11T00:00:00,2018-12-04T00:00:00,MEDIUM,BRISBANE,-27.402357,153.056983,Supermarket
800043,CREDIT,Taylor Grocery,David,Taylor,1978-01-23T00:00:00,2019-07-15T00:00:00,MEDIUM,BRISBANE,-27.473628,152.955089,Grocery


In [36]:
account_FROMACCT = account.copy()
account_FROMACCT.columns = ['FROMACCT_' + str(col) for col in account_FROMACCT.columns]
account_FROMACCT.head()

,FROMACCT_ACCTTYPE,FROMACCT_ACCTNAME,FROMACCT_FIRSTNAME,FROMACCT_SURNAME,FROMACCT_DOB,FROMACCT_ACCTCREATED,FROMACCT_ACCTRISK,FROMACCT_ADDRESS,FROMACCT_ADDRESS_LAT,FROMACCT_ADDRESS_LON,FROMACCT_CATEGORY
ACCTID,,,,,,,,,,,
800039,BUSINESS,Young Supermarket,Kevin,Young,1982-02-23T00:00:00,2018-09-20T00:00:00,MEDIUM,BRISBANE,-27.512179,153.073105,Supermarket
800040,BUSINESS,Williams Grocery,Lindsay,Williams,1959-02-20T00:00:00,2018-02-24T00:00:00,MEDIUM,BRISBANE,-27.504303,153.073470,Grocery
800041,CREDIT,Myers Supermarket,Lisa,Myers,1982-09-04T00:00:00,2018-01-11T00:00:00,HIGH,BRISBANE,-27.484970,153.097588,Supermarket
800042,BUSINESS,Martinez Supermarket,Brittany,Martinez,1976-10-11T00:00:00,2018-12-04T00:00:00,MEDIUM,BRISBANE,-27.402357,153.056983,Supermarket
800043,CREDIT,Taylor Grocery,David,Taylor,1978-01-23T00:00:00,2019-07-15T00:00:00,MEDIUM,BRISBANE,-27.473628,152.955089,Grocery


In [37]:
account_TOACCT = account.copy()
account_TOACCT.columns = ['TOACCT_' + str(col) for col in account_TOACCT.columns]
account_TOACCT.head()

,TOACCT_ACCTTYPE,TOACCT_ACCTNAME,TOACCT_FIRSTNAME,TOACCT_SURNAME,TOACCT_DOB,TOACCT_ACCTCREATED,TOACCT_ACCTRISK,TOACCT_ADDRESS,TOACCT_ADDRESS_LAT,TOACCT_ADDRESS_LON,TOACCT_CATEGORY
ACCTID,,,,,,,,,,,
800039,BUSINESS,Young Supermarket,Kevin,Young,1982-02-23T00:00:00,2018-09-20T00:00:00,MEDIUM,BRISBANE,-27.512179,153.073105,Supermarket
800040,BUSINESS,Williams Grocery,Lindsay,Williams,1959-02-20T00:00:00,2018-02-24T00:00:00,MEDIUM,BRISBANE,-27.504303,153.073470,Grocery
800041,CREDIT,Myers Supermarket,Lisa,Myers,1982-09-04T00:00:00,2018-01-11T00:00:00,HIGH,BRISBANE,-27.484970,153.097588,Supermarket
800042,BUSINESS,Martinez Supermarket,Brittany,Martinez,1976-10-11T00:00:00,2018-12-04T00:00:00,MEDIUM,BRISBANE,-27.402357,153.056983,Supermarket
800043,CREDIT,Taylor Grocery,David,Taylor,1978-01-23T00:00:00,2019-07-15T00:00:00,MEDIUM,BRISBANE,-27.473628,152.955089,Grocery


In [38]:
transaction = pd.read_csv('anomaly-detection_anomaly-detection/ftxn2.txt')
transaction.set_index('FROMACCTID',inplace=True)
transaction.head()

,TXN_ID,TXTYPE,AMOUNT,TOACCTID,TXDATE,REFERENCE,ISFRAUD,ISFLAGGED
FROMACCTID,,,,,,,,
500607,950060711303,PAYMENT,320,800242,2020-01-22T17:00:00,Dinner with Shane,N,Y
500607,950060711401,PAYMENT,48,800111,2020-01-23T08:00:00,Breakfast with Steven,N,Y
500607,950060711402,TRANSFER,6209,500929,2020-01-23T13:00:00,Holiday Cost,N,Y
500607,950060711403,PAYMENT,66,800252,2020-01-23T17:00:00,Meal with Mary,N,Y
500607,950060711501,PAYMENT,47,800116,2020-01-24T08:00:00,Coffee with Brian,N,Y


In [39]:
transaction = transaction.join(account_FROMACCT, how='left')
transaction.head()

,TXN_ID,TXTYPE,AMOUNT,TOACCTID,TXDATE,REFERENCE,ISFRAUD,ISFLAGGED,FROMACCT_ACCTTYPE,FROMACCT_ACCTNAME,FROMACCT_FIRSTNAME,FROMACCT_SURNAME,FROMACCT_DOB,FROMACCT_ACCTCREATED,FROMACCT_ACCTRISK,FROMACCT_ADDRESS,FROMACCT_ADDRESS_LAT,FROMACCT_ADDRESS_LON,FROMACCT_CATEGORY
500000,950000007202,TRANSFER,7479,500417,2019-12-12T13:00:00,Subscription Costs,N,Y,MAXI-SAVER,"Gutierrez, Jenna",Jenna,Gutierrez,1963-10-18T00:00:00,2015-03-28T00:00:00,LOW,BRISBANE,-27.501761,153.038465,CUSTOMER
500000,950000007203,PAYMENT,465,800242,2019-12-12T17:00:00,Meal with Jose,N,Y,MAXI-SAVER,"Gutierrez, Jenna",Jenna,Gutierrez,1963-10-18T00:00:00,2015-03-28T00:00:00,LOW,BRISBANE,-27.501761,153.038465,CUSTOMER
500000,950000007301,PAYMENT,69,800195,2019-12-13T08:00:00,Breakfast with Whitney,N,Y,MAXI-SAVER,"Gutierrez, Jenna",Jenna,Gutierrez,1963-10-18T00:00:00,2015-03-28T00:00:00,LOW,BRISBANE,-27.501761,153.038465,CUSTOMER
500000,950000007302,TRANSFER,9763,500042,2019-12-13T13:00:00,Settling Deal,N,Y,MAXI-SAVER,"Gutierrez, Jenna",Jenna,Gutierrez,1963-10-18T00:00:00,2015-03-28T00:00:00,LOW,BRISBANE,-27.501761,153.038465,CUSTOMER
500000,950000007303,PAYMENT,443,800271,2019-12-13T17:00:00,Dinner with Samuel,N,Y,MAXI-SAVER,"Gutierrez, Jenna",Jenna,Gutierrez,1963-10-18T00:00:00,2015-03-28T00:00:00,LOW,BRISBANE,-27.501761,153.038465,CUSTOMER


In [40]:
transaction.reset_index(inplace=True)
transaction.rename(columns={'index':'FROMACCTID'},inplace=True)
transaction.head()

,FROMACCTID,TXN_ID,TXTYPE,AMOUNT,TOACCTID,TXDATE,REFERENCE,ISFRAUD,ISFLAGGED,FROMACCT_ACCTTYPE,FROMACCT_ACCTNAME,FROMACCT_FIRSTNAME,FROMACCT_SURNAME,FROMACCT_DOB,FROMACCT_ACCTCREATED,FROMACCT_ACCTRISK,FROMACCT_ADDRESS,FROMACCT_ADDRESS_LAT,FROMACCT_ADDRESS_LON,FROMACCT_CATEGORY
0,500000,950000007202,TRANSFER,7479,500417,2019-12-12T13:00:00,Subscription Costs,N,Y,MAXI-SAVER,"Gutierrez, Jenna",Jenna,Gutierrez,1963-10-18T00:00:00,2015-03-28T00:00:00,LOW,BRISBANE,-27.501761,153.038465,CUSTOMER
1,500000,950000007203,PAYMENT,465,800242,2019-12-12T17:00:00,Meal with Jose,N,Y,MAXI-SAVER,"Gutierrez, Jenna",Jenna,Gutierrez,1963-10-18T00:00:00,2015-03-28T00:00:00,LOW,BRISBANE,-27.501761,153.038465,CUSTOMER
2,500000,950000007301,PAYMENT,69,800195,2019-12-13T08:00:00,Breakfast with Whitney,N,Y,MAXI-SAVER,"Gutierrez, Jenna",Jenna,Gutierrez,1963-10-18T00:00:00,2015-03-28T00:00:00,LOW,BRISBANE,-27.501761,153.038465,CUSTOMER
3,500000,950000007302,TRANSFER,9763,500042,2019-12-13T13:00:00,Settling Deal,N,Y,MAXI-SAVER,"Gutierrez, Jenna",Jenna,Gutierrez,1963-10-18T00:00:00,2015-03-28T00:00:00,LOW,BRISBANE,-27.501761,153.038465,CUSTOMER
4,500000,950000007303,PAYMENT,443,800271,2019-12-13T17:00:00,Dinner with Samuel,N,Y,MAXI-SAVER,"Gutierrez, Jenna",Jenna,Gutierrez,1963-10-18T00:00:00,2015-03-28T00:00:00,LOW,BRISBANE,-27.501761,153.038465,CUSTOMER


In [41]:
transaction.set_index('TOACCTID',inplace=True)
transaction.head()

,FROMACCTID,TXN_ID,TXTYPE,AMOUNT,TXDATE,REFERENCE,ISFRAUD,ISFLAGGED,FROMACCT_ACCTTYPE,FROMACCT_ACCTNAME,FROMACCT_FIRSTNAME,FROMACCT_SURNAME,FROMACCT_DOB,FROMACCT_ACCTCREATED,FROMACCT_ACCTRISK,FROMACCT_ADDRESS,FROMACCT_ADDRESS_LAT,FROMACCT_ADDRESS_LON,FROMACCT_CATEGORY
TOACCTID,,,,,,,,,,,,,,,,,,,
500417,500000,950000007202,TRANSFER,7479,2019-12-12T13:00:00,Subscription Costs,N,Y,MAXI-SAVER,"Gutierrez, Jenna",Jenna,Gutierrez,1963-10-18T00:00:00,2015-03-28T00:00:00,LOW,BRISBANE,-27.501761,153.038465,CUSTOMER
800242,500000,950000007203,PAYMENT,465,2019-12-12T17:00:00,Meal with Jose,N,Y,MAXI-SAVER,"Gutierrez, Jenna",Jenna,Gutierrez,1963-10-18T00:00:00,2015-03-28T00:00:00,LOW,BRISBANE,-27.501761,153.038465,CUSTOMER
800195,500000,950000007301,PAYMENT,69,2019-12-13T08:00:00,Breakfast with Whitney,N,Y,MAXI-SAVER,"Gutierrez, Jenna",Jenna,Gutierrez,1963-10-18T00:00:00,2015-03-28T00:00:00,LOW,BRISBANE,-27.501761,153.038465,CUSTOMER
500042,500000,950000007302,TRANSFER,9763,2019-12-13T13:00:00,Settling Deal,N,Y,MAXI-SAVER,"Gutierrez, Jenna",Jenna,Gutierrez,1963-10-18T00:00:00,2015-03-28T00:00:00,LOW,BRISBANE,-27.501761,153.038465,CUSTOMER
800271,500000,950000007303,PAYMENT,443,2019-12-13T17:00:00,Dinner with Samuel,N,Y,MAXI-SAVER,"Gutierrez, Jenna",Jenna,Gutierrez,1963-10-18T00:00:00,2015-03-28T00:00:00,LOW,BRISBANE,-27.501761,153.038465,CUSTOMER


In [42]:
transaction = transaction.join(account_TOACCT, how='left')
transaction

,FROMACCTID,TXN_ID,TXTYPE,AMOUNT,TXDATE,REFERENCE,ISFRAUD,ISFLAGGED,FROMACCT_ACCTTYPE,FROMACCT_ACCTNAME,...,TOACCT_ACCTNAME,TOACCT_FIRSTNAME,TOACCT_SURNAME,TOACCT_DOB,TOACCT_ACCTCREATED,TOACCT_ACCTRISK,TOACCT_ADDRESS,TOACCT_ADDRESS_LAT,TOACCT_ADDRESS_LON,TOACCT_CATEGORY
500000,500000,950000030602,TRANSFER,1482,2020-08-02T13:00:00,Gift Costs,N,Y,MAXI-SAVER,"Gutierrez, Jenna",...,"Gutierrez, Jenna",Jenna,Gutierrez,1963-10-18T00:00:00,2015-03-28T00:00:00,LOW,BRISBANE,-27.501761,153.038465,CUSTOMER
500000,500000,950000035202,TRANSFER,3452,2020-09-17T13:00:00,Payment for Job,N,Y,MAXI-SAVER,"Gutierrez, Jenna",...,"Gutierrez, Jenna",Jenna,Gutierrez,1963-10-18T00:00:00,2015-03-28T00:00:00,LOW,BRISBANE,-27.501761,153.038465,CUSTOMER
500000,500003,950000327102,TRANSFER,4972,2020-06-28T13:00:00,Settling Deal,N,Y,SAVINGS,"Burgess, Shelley",...,"Gutierrez, Jenna",Jenna,Gutierrez,1963-10-18T00:00:00,2015-03-28T00:00:00,LOW,BRISBANE,-27.501761,153.038465,CUSTOMER
500000,500006,950000631002,TRANSFER,1230,2020-08-06T13:00:00,Gift Costs,N,Y,CREDIT,"Dennis, Thomas",...,"Gutierrez, Jenna",Jenna,Gutierrez,1963-10-18T00:00:00,2015-03-28T00:00:00,LOW,BRISBANE,-27.501761,153.038465,CUSTOMER
500000,500007,950000722102,TRANSFER,3354,2020-05-09T13:00:00,Gift Costs,N,Y,CREDIT,"Wagner, Nicole",...,"Gutierrez, Jenna",Jenna,Gutierrez,1963-10-18T00:00:00,2015-03-28T00:00:00,LOW,BRISBANE,-27.501761,153.038465,CUSTOMER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
800427,800412,9150080115010,TRANSFER,200018,2020-02-28T10:00:00,Water Costs,N,Y,BUSINESS,Price Consulting,...,Sanders Finance,Randy,Sanders,1960-09-25T00:00:00,2017-07-14T00:00:00,MEDIUM,BRISBANE,-27.429083,152.960128,Finance
800434,800427,9150080015011,TRANSFER,200009,2020-02-28T10:00:00,Settling Deal,N,Y,BUSINESS,Sanders Finance,...,Sutton Trading,Kathleen,Sutton,1985-03-12T00:00:00,2015-08-27T00:00:00,MEDIUM,BRISBANE,-27.492489,153.025572,Trading
800434,800427,9150080115011,TRANSFER,200020,2020-02-28T10:00:00,Subscription Costs,N,Y,BUSINESS,Sanders Finance,...,Sutton Trading,Kathleen,Sutton,1985-03-12T00:00:00,2015-08-27T00:00:00,MEDIUM,BRISBANE,-27.492489,153.025572,Trading
800435,800434,9150080015012,TRANSFER,200003,2020-02-28T10:00:00,Holiday Cost,N,Y,CREDIT,Sutton Trading,...,Johnson Brewery,Stephanie,Johnson,1973-04-03T00:00:00,2017-12-01T00:00:00,LOW,BRISBANE,-27.459187,153.082077,Brewery


In [43]:
transaction.reset_index(inplace=True)
transaction.rename(columns={'index':'TOACCTID'},inplace=True)
transaction.head()

,TOACCTID,FROMACCTID,TXN_ID,TXTYPE,AMOUNT,TXDATE,REFERENCE,ISFRAUD,ISFLAGGED,FROMACCT_ACCTTYPE,...,TOACCT_ACCTNAME,TOACCT_FIRSTNAME,TOACCT_SURNAME,TOACCT_DOB,TOACCT_ACCTCREATED,TOACCT_ACCTRISK,TOACCT_ADDRESS,TOACCT_ADDRESS_LAT,TOACCT_ADDRESS_LON,TOACCT_CATEGORY
0,500000,500000,950000030602,TRANSFER,1482,2020-08-02T13:00:00,Gift Costs,N,Y,MAXI-SAVER,...,"Gutierrez, Jenna",Jenna,Gutierrez,1963-10-18T00:00:00,2015-03-28T00:00:00,LOW,BRISBANE,-27.501761,153.038465,CUSTOMER
1,500000,500000,950000035202,TRANSFER,3452,2020-09-17T13:00:00,Payment for Job,N,Y,MAXI-SAVER,...,"Gutierrez, Jenna",Jenna,Gutierrez,1963-10-18T00:00:00,2015-03-28T00:00:00,LOW,BRISBANE,-27.501761,153.038465,CUSTOMER
2,500000,500003,950000327102,TRANSFER,4972,2020-06-28T13:00:00,Settling Deal,N,Y,SAVINGS,...,"Gutierrez, Jenna",Jenna,Gutierrez,1963-10-18T00:00:00,2015-03-28T00:00:00,LOW,BRISBANE,-27.501761,153.038465,CUSTOMER
3,500000,500006,950000631002,TRANSFER,1230,2020-08-06T13:00:00,Gift Costs,N,Y,CREDIT,...,"Gutierrez, Jenna",Jenna,Gutierrez,1963-10-18T00:00:00,2015-03-28T00:00:00,LOW,BRISBANE,-27.501761,153.038465,CUSTOMER
4,500000,500007,950000722102,TRANSFER,3354,2020-05-09T13:00:00,Gift Costs,N,Y,CREDIT,...,"Gutierrez, Jenna",Jenna,Gutierrez,1963-10-18T00:00:00,2015-03-28T00:00:00,LOW,BRISBANE,-27.501761,153.038465,CUSTOMER


In [44]:
transaction.shape

(1533066, 31)